# Import

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib

In [ ]:
# plotting options 
font = {'size'   : 20}
plt.rc('font', **font)
plt.rc('text', usetex=True)

matplotlib.rc('figure', figsize=(18, 6) )

## Define channel matrix (TODO)

$P_{xy} = \begin{pmatrix}
    P(y_1|x_1) & P(y_1|x_2) & \cdots\\
    P(y_2|x_1) & P(y_2|x_2) & \cdots\\
    \vdots & \vdots & \ddots
\end{pmatrix}$

In [ ]:
switch = 'tut_problem_4'  

if switch == 'bsc':
    X = np.arange(2) 
    Y = np.arange(2)     
    
    delta = .2
    P_yx = np.array([
        [1-delta, delta],
        [delta, 1-delta]
    ])
elif switch == 'bec':
    X = np.arange(2) 
    Y = np.arange(3)

    delta = .2
    P_yx = np.array([
        [1-delta, 0],
        [delta, delta],
        [0, 1-delta]
    ])
elif switch == 'tut_problem_4':

    X = np.arange( 3 )
    Y = np.arange( 3 )

    # P_yx = np.array([
    #     # TODO
    # ])

print( P_yx )


## Helper functions for Blahut-Arimoto

In [ ]:
# getting Q given P and p_X
def get_Q_xy( P, p_X ):
    '''
        determines Q as provided by Blahut-Arimoto

        IN: P, p_X
        OUT: Q
    '''
    # init Q as |X| x |Y| matrix
    Q = np.zeros( np.shape(P)[::-1] )

    for x in X:
        for y in Y:
            Q_denom_y = (P @ p_X )[y]

            Q[ x, y ] = p_X[ x ] * P_yx[ y, x ] / Q_denom_y

    return Q

# getting p_X given P and Q
def get_p_X( P, Q ):

    # init p_X as |X| vector
    p_X = np.ones( np.shape( P ) )[1]

    # find denominator
    denom = 0
    for x in X:
        prod = 1
        for y in Y:
            prod *= Q[ x, y ]**P[ y, x ]
        denom += prod
    
    # get P(x)
    for x in X:
        prod = 1
        for y in Y:
            prod *= Q[ x, y ]**P[ y, x ]

        p_X[ x ] = prod / denom
    
    return p_X

## Actual algorithm of Blahut-Arimoto (TODO)

In [ ]:
# define Blahut-Arimoto Algorithm
def Blahut_Arimoto( P, max_iterations = 1e2 ):
    '''
        performs alg. of Blahut-Arimoto

        IN: P_yx
        OUT: p_X_max, C
    '''
    # initial distribution
    p_X = np.empty( ( 1, X.size ) )
    # p_X[ 0, : ] = [1/3, 1/3, 1/3] # uniform distribution over 3 inputs (example)
    p_X[ 0, : ] = [
        #TODO define initial input distribution
    ]
    
    i = 0

    # loop for a max. number of times (and stop if p has not changed)
    while i < max_iterations:

        # get Q
        Q = get_Q_xy( P, p_X[ i ])

        # get new p_X
        p_X_new = get_p_X( P_yx, Q )

        # append probabilities as row to p_X
        p_X = np.vstack( [ p_X, p_X_new ] )

        # increase counter
        i += 1

    return p_X

# Comparison of Blahut-Arimoto to differential evolution

In [ ]:
def calc_MI(P_YgivenX, p_X):
    """ get the mutual information I(X;Y) resulting from a given P(y|x) and P(x)"""
    p_Y = P_YgivenX @ p_X
    nonzero = p_Y != 0
    H_Y = -np.sum(p_Y[nonzero] * np.log2(p_Y[nonzero]))

    nonzero = (P_YgivenX != 0) & (p_X != 0)
    H_YgivenX = -np.sum((P_YgivenX * p_X)[nonzero] * np.log2(P_YgivenX[nonzero]))

    return H_Y - H_YgivenX


p_X = Blahut_Arimoto(P_yx, 50)

for iteration in [0, 1, 2, 10, 50]:
    nice_probabilities = ', '.join([f'P(x_{i+1}) = {p:.4f}' for i, p in enumerate(p_X[iteration, :])])
    print(f'Iteration {iteration}: {nice_probabilities}')
print()

p_maximizer_BA = p_X[ -1, : ]
C_BA = calc_MI(P_yx, p_maximizer_BA)
print(f'Capacity according to Blahut-Arimoto: {C_BA:.4f} bit')